# 🚀 UNSW-NB15 Analysis - Google Colab Version

Bu notebook Google Colab'da çalıştırmak için optimize edilmiştir.

**Adımlar:**
1. Projeyi GitHub'dan klonla
2. Bağımlılıkları yükle
3. Veri setini indir
4. Analizi çalıştır

---

## 1️⃣ Google Drive'a Bağlan (Opsiyonel - Sonuçları kaydetmek için)

In [ ]:
# Google Drive'a bağlan (sonuçları kaydetmek için)
from google.colab import drive
drive.mount('/content/drive')

# Çalışma dizini
import os
os.chdir('/content')
print("✓ Google Drive bağlandı")

## 2️⃣ Projeyi GitHub'dan Klonla

In [ ]:
# Projeyi klonla
!git clone https://github.com/sedahacettepetez-pixel/sibermakale.git
%cd sibermakale

# Branch'e geç
!git checkout claude/unsw-nb15-setup-config-01DEmKoC2eHKvoYkAuYHsr8a

!ls -la
print("✓ Proje klonlandı")

## 3️⃣ Sistem Bilgisi

In [ ]:
# GPU kontrolü
!nvidia-smi

# Python versiyon
!python --version

# Bellek durumu
!free -h

## 4️⃣ Bağımlılıkları Yükle

In [ ]:
# pip güncelle
!pip install --upgrade pip

# Bağımlılıkları yükle (bu 5-10 dakika sürebilir)
!pip install -q -r requirements.txt

print("✓ Tüm paketler yüklendi")

## 5️⃣ UNSW-NB15 Veri Setini İndir

### Yöntem 1: Kaggle API ile (Önerilen)

In [ ]:
# Kaggle API token yükle
# 1. Kaggle'dan kaggle.json indir: https://www.kaggle.com/settings/account
# 2. Aşağıdaki hücreyi çalıştır ve dosyayı yükle

from google.colab import files
uploaded = files.upload()  # kaggle.json dosyasını seç

# Kaggle dizini oluştur
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✓ Kaggle API hazır")

In [ ]:
# Veri setini indir
!kaggle datasets download -d mrwellsdavid/unsw-nb15

# Çıkart
!unzip -q unsw-nb15.zip -d data/

# Kontrol et
!ls -lh data/

print("✓ Veri seti hazır")

### Yöntem 2: Manuel Yükleme (Alternatif)

In [ ]:
# Eğer Kaggle API çalışmazsa, manuel olarak yükleyin
from google.colab import files

print("Lütfen UNSW_NB15_training-set.csv ve UNSW_NB15_testing-set.csv dosyalarını yükleyin")
uploaded = files.upload()

# data/ klasörüne taşı
!mkdir -p data
!mv UNSW*.csv data/

print("✓ Dosyalar yüklendi")

## 6️⃣ Konfigürasyonu Yükle ve Kontrol Et

In [ ]:
import json
import pandas as pd
import numpy as np

# Config yükle
with open('config.json', 'r') as f:
    config = json.load(f)

print("✓ Konfigürasyon yüklendi")
print(f"Proje: {config['project']['name']}")
print(f"Random Seed: {config['project']['seed']}")

## 7️⃣ Ana Analizi Çalıştır

Ana notebook'u açmak için:
1. Sol taraftaki dosya menüsünden `unsw_nb15_analysis.ipynb` dosyasını açın
2. Veya aşağıdaki hücreleri çalıştırarak temel analizleri burada yapın

In [ ]:
# Utility fonksiyonlarını import et
from utils import (
    load_config, ensure_directories,
    create_data_inventory, create_eda_overview
)

# Dizinleri oluştur
ensure_directories(config)

print("✓ Sistem hazır")

In [ ]:
# Veri yükle ve ilk bakış
train_path = config['data']['train_path']
test_path = config['data']['test_path']

print("Veri yükleniyor...")
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

df_train['split'] = 'train'
df_test['split'] = 'test'
df = pd.concat([df_train, df_test], ignore_index=True)

print(f"✓ Veri yüklendi: {len(df):,} satır, {len(df.columns)} sütun")
print(f"  Train: {len(df_train):,}")
print(f"  Test: {len(df_test):,}")

df.head()

In [ ]:
# Hedef dağılımı
import matplotlib.pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Binary
df['label'].value_counts().plot(kind='bar', ax=axes[0], color=['green', 'red'])
axes[0].set_title('Binary Label (0=Normal, 1=Attack)')
axes[0].set_xlabel('Label')

# Multi-class
df['attack_cat'].value_counts().plot(kind='barh', ax=axes[1])
axes[1].set_title('Attack Categories')

plt.tight_layout()
plt.show()

print("\nBinary Distribution:")
print(df['label'].value_counts())
print("\nAttack Categories:")
print(df['attack_cat'].value_counts())

## 8️⃣ Model Eğitimi (Hızlı Demo)

Hızlı bir model eğitimi örneği:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Basit feature selection
numeric_features = ['dur', 'sbytes', 'dbytes', 'spkts', 'dpkts']

# Veri hazırlama
X = df[numeric_features].fillna(0)
y = df['attack_cat']

# Label encoding
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
)

print(f"✓ Train: {len(X_train):,} | Test: {len(X_test):,}")

# Model eğitimi
print("\nModel eğitiliyor (bu birkaç dakika sürebilir)...")
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, n_jobs=-1)
clf.fit(X_train, y_train)

# Tahmin
y_pred = clf.predict(X_test)

# Sonuçlar
print("\n" + "="*50)
print("HIZLI MODEL SONUÇLARI (Random Forest)")
print("="*50)
print(classification_report(y_test, y_pred, target_names=le.classes_, zero_division=0))

print("\n✓ Hızlı demo tamamlandı!")
print("\nDaha detaylı analiz için 'unsw_nb15_analysis.ipynb' dosyasını açın.")

## 9️⃣ Sonuçları Kaydet (Opsiyonel)

In [ ]:
# Sonuçları Drive'a kaydet
!cp -r artifacts/ /content/drive/MyDrive/UNSW_NB15_Results/

print("✓ Sonuçlar Google Drive'a kaydedildi: MyDrive/UNSW_NB15_Results/")

## 🔟 Sonuçları İndir

In [ ]:
# Sonuçları ZIP olarak indir
!zip -r results.zip artifacts/

from google.colab import files
files.download('results.zip')

print("✓ Sonuçlar indiriliyor...")

---

## 📚 Sonraki Adımlar

1. **Detaylı Analiz**: `unsw_nb15_analysis.ipynb` notebook'unu açın
2. **Model Karşılaştırması**: LightGBM, XGBoost, TabTransformer
3. **Hiperparametre Optimizasyonu**: Optuna ile HPO
4. **Sonuçları Paylaş**: Google Drive'da sakla

---

**Hazırlayan:** UNSW-NB15 Research Team  
**Tarih:** 2025-11-15  
**Colab Uyumlu Versiyon**

🚀 **Happy Analyzing!**